In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

pio.templates.default = "plotly_white"

In [2]:
data = pd.read_csv("data.csv", index_col=0, parse_dates=True)

In [3]:
data

,Update Duration (ms),Reading Date and Time (ISO),Air pressure (mb),Air temperature (C),Tide height (m),Wind direction (deg),Wind gust speed (kn),Wind speed (kn),True air temperature (C),True tide height (m),Independent tide height prediction (m),Independent tide height deviation (m),Dependent tide height prediction (m),Dependent tide height deviation (m),Independent air temperature prediction (C),Independent air temperature deviation (C),Dependent air temperature prediction (C),Dependent air temperature deviation (C)
Update Date and Time (ISO),,,,,,,,,,,,,,,,,,
2007-05-26 12:05:00,0,2007-05-26T12:05:00,NaN,NaN,NaN,NaN,NaN,NaN,16.92,2.30,2.4007,0.039110,2.3715,0.036670,17.2348,0.23222,17.2339,0.23187
2007-05-26 12:10:00,0,2007-05-26T12:10:00,1006.0,16.7,2.3,22.0,12.0,12.4,16.42,2.24,2.4016,0.058707,2.3365,0.045564,17.1932,0.29931,17.1848,0.29732
2007-05-26 12:15:00,0,2007-05-26T12:15:00,1006.0,16.0,2.2,13.0,18.1,13.2,16.00,2.19,2.2945,0.037006,2.2836,0.029853,16.8005,0.25186,16.8130,0.24920
2007-05-26 12:20:00,0,2007-05-26T12:20:00,1006.0,15.9,2.1,16.0,14.2,13.0,15.92,2.14,2.1455,0.021627,2.1553,0.019945,16.0734,0.17207,16.0621,0.17101
2007-05-26 12:25:00,0,2007-05-26T12:25:00,1005.0,16.1,2.1,12.0,14.4,12.0,16.08,2.09,2.0900,0.028609,2.1115,0.023468,15.8401,0.21507,15.8126,0.21158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007-05-31 15:55:00,0,2007-05-31T15:55:00,NaN,NaN,NaN,NaN,NaN,NaN,16.00,2.19,2.1759,0.020713,2.2065,0.018579,16.1252,0.15125,16.1222,0.14964
2007-05-31 16:00:00,0,2007-05-31T16:00:00,1007.0,15.9,2.1,123.0,22.3,20.4,15.92,2.08,2.0803,0.019198,2.0998,0.017608,16.0285,0.14422,16.0271,0.14289
2007-05-31 16:05:00,0,2007-05-31T16:05:00,1007.0,15.6,2.0,119.0,21.6,19.8,15.67,1.98,1.9782,0.023006,1.9904,0.020269,15.9775,0.16485,15.9777,0.16287


In [4]:
px.line(
    data,
    y=[ "True tide height (m)", "Tide height (m)",],
)

# Questions

In [5]:
def plot_gp(x, y, u):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, mode="markers+lines", name="Predictions"))
    # filled area
    fig.add_trace(
        go.Scatter(
            x=x, y=y + u, fill=None, line_color="blue", line_width=0, showlegend=False
        )
    )
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y - u,
            fill="tonexty",
            line_color="blue",
            line_width=0,
            showlegend=False,
        )
    )
    return fig

In [6]:
np.random.seed(42)
time_norm = 1 / (3600 * 1e9)  # ns to hour
t, y = (
    data["Tide height (m)"].dropna().index.astype(int).to_numpy().reshape(-1, 1)
    * time_norm,
    data["Tide height (m)"].dropna(),
) # indices where there is data
x = data.index.astype(int).to_numpy() * time_norm # all indices
x = x - x[0]  # normalise to start from 0
t = t - t[0]  # normalise to start from 0
x

array([0.00000000e+00, 8.33333333e-02, 1.66666667e-01, ...,
       1.24000000e+02, 1.24083333e+02, 1.24166667e+02])

## Questions 1-4

In [7]:
def kernel_function(s, t, hparams=None):
    # small reshape hack to vectorise the operations
    s = s.reshape(-1, 1)
    t = t.reshape(1, -1)
    if hparams is None:
        # parameters, found by experimentation
        theta = 2
        tau = 12
        sigma = 1

        phi = 1
        eta = 1

        zeta = 0.01
    else:
        theta, tau, sigma, phi, eta, zeta = hparams

    # Adding kernels together
    A = theta**2 * np.exp(-2 / (sigma**2) * (np.sin(np.pi * (s - t) / tau)) ** 2)
    B = phi**2 * np.exp(-((s - t) ** 2) / (2 * eta**2))
    C = zeta**2 * (s == t)

    return B+A+C

In [8]:
cov_kernel = kernel_function(x, x)
mean = np.zeros((cov_kernel.shape[0],))

# We can sample our GP function from a multivariate normal
samples = np.random.multivariate_normal(mean, cov_kernel, 3).T
fig =  px.line(pd.DataFrame(samples, index=x))
fig.update_layout(xaxis_title='Hour', yaxis_title='Tide height (m)', title='Samples from Prior', showlegend=False)

In [9]:
def get_mean_and_cov(hparams=None):
    # we calculate our posterior distribution conditioned on data
    K = kernel_function(x, x, hparams)
    Kxx = kernel_function(t, t, hparams)
    Kx = kernel_function(x, t, hparams)
    Kxx_inv = np.linalg.pinv(Kxx)

    # mean and cov of conditioned GP
    mean = Kx @ Kxx_inv @ y
    cov = K - Kx @ Kxx_inv @ Kx.T
    return mean, cov

mean, cov = get_mean_and_cov()
# Get samples from the posterior
samples = np.random.multivariate_normal(mean, cov, 3).T

# we plot the data with the extrapolation
fig = px.line(pd.DataFrame(samples, index=x))
fig.update_layout(xaxis_title='Hour', yaxis_title='Tide height (m)', title='Samples from Posterior')

In [10]:
fig = plot_gp(x, mean, np.diag(abs(cov)) ** (1/2))
fig.add_trace(
    go.Scatter(x=x, y=data["True tide height (m)"], name="True tide height (m)")
)
fig.update_layout(xaxis_title='Hour', yaxis_title='Tide height (m)', title='Posterior prediction and deviation')

In [11]:
# MSE
np.mean((data["True tide height (m)"] - mean) ** 2)

np.float64(0.030015527077436814)

In [12]:
# comparing with exisiting predictions
np.mean(
    (data["Independent tide height prediction (m)"] - data["True tide height (m)"]) ** 2
)

np.float64(0.003624391605723371)

## Question 5

MLE estimation of hyperparameters
$$\theta_{MLE} = \argmax P(D | \theta) $$

Where the likelihood is defined as 
$$
p(y|X,\theta) = \mathcal{N}(y|0, K(X,X|\theta))\\
$$
We can take the log instead of the straight likelihood for easier optimisation.
$$
\log p(y|X,\theta) = -\frac{1}{2} y^TK^{-1}y - \frac{1}{2}\log|K| -\frac{n}{2}\log 2\pi
$$

MAP estimation of hyperparameters
$$ P(\theta | D) = \frac{P(D | \theta) P(\theta)}  {P(D)}\\
\theta_{MAP} = \argmax P(\theta | D)\\
$$
We use the same strategy and use the log of the MAP
$$
P(\theta | D) = -\frac{1}{2} y^TK^{-1}y - \frac{1}{2}\log|K| -\frac{n}{2}\log 2\pi + \log p(\theta) - p(D)
$$ 
As priors, we can assume a uniform distribution between 0 and 20 for each of the parameters. This just means our MAP will be -$\infty$ outside those ranges, it is similar to putting a constraint in our optimisation.
$$ 
U(x) = 
\begin{cases}
\frac{1}{20} \qquad x \in ]0,20]\\
0 \qquad \text{otherwise}
\end{cases}
$$
Since we are taking logarithms, we don't want to include 0 in the interval. The evidence will be a constant given the data so we can safely not compute it as we are optimising.

In [13]:
# We can use scipy to optimise this
from scipy import optimize


def log_likelihood(hparams):
    jitter = 1 # to make sure the determinant is not 0
    K = kernel_function(t, t, hparams)
    a = -0.5 * y.T @ np.linalg.pinv(K) @ y
    b = -0.5 * np.log(np.linalg.det(K + jitter * np.eye(K.shape[0])))
    c = -len(t) / 2 * np.log(2 * np.pi)
    return -(a + b + c)


log_likelihood([2, 12, 1, 1,1 , 0.1])


np.float64(998.8426327338052)

In [14]:
res = optimize.minimize(
    log_likelihood,
    x0=[2, 12, 1, 1,1 , 0.1],
    # method="L-BFGS-B",
    tol = 1e-10
)
res

/Users/theo/Library/Caches/pypoetry/virtualenvs/aims-vYD_rrQz-py3.9/lib/python3.9/site-packages/numpy/linalg/_linalg.py:2371: RuntimeWarning:

overflow encountered in det

/Users/theo/Library/Caches/pypoetry/virtualenvs/aims-vYD_rrQz-py3.9/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:590: RuntimeWarning:

invalid value encountered in subtract



  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 923.2009938671487
        x: [ 2.355e+00  1.236e+01  1.456e+00  2.563e-01  9.645e-01
             1.750e-01]
      nit: 24
      jac: [ 7.553e-04  3.281e-04  8.316e-04  7.935e-04  1.266e-03
             8.087e-04]
 hess_inv: [[ 3.447e-01  3.088e-02 ... -1.242e-01  1.189e-02]
            [ 3.088e-02  2.991e-03 ... -1.134e-02  1.043e-03]
            ...
            [-1.242e-01 -1.134e-02 ...  5.452e-02 -4.123e-03]
            [ 1.189e-02  1.043e-03 ... -4.123e-03  8.189e-04]]
     nfev: 589
     njev: 82

In [15]:
mean, cov = get_mean_and_cov(res.x)
res.x, np.mean((data["True tide height (m)"] - mean) ** 2)

(array([ 2.35531925, 12.36125375,  1.45644993,  0.25632512,  0.96447411,
         0.17498519]),
 np.float64(0.004316088688220089))

In [16]:
fig = plot_gp(x, mean, np.diag(cov)**0.5)
fig.add_trace(
    go.Scatter(x=x, y=data["True tide height (m)"], name="True tide height (m)")
)
fig.update_layout(xaxis_title='Hour', yaxis_title='Tide height (m)', title='Predictions with optimised hyperparameters with MLE')

In [17]:
# MAP estimate
def map_estimate(hparams):
    jitter = 1
    K = kernel_function(t, t, hparams)
    a = -0.5 * y.T @ np.linalg.pinv(K) @ y
    b = -0.5 * np.log(np.linalg.det(K + jitter * np.eye(K.shape[0])))
    c = -len(t) / 2 * np.log(2 * np.pi)
    # hparam prior terms
    hparam_prior = 0
    # terrible implementation
    for hp in hparams:
        if 0 < hp <= 20:
            hparam_prior += np.log(1/20)
        else:
            hparam_prior += np.log(10**(-10)) # small number to not break 
    # take the negative as we are minimizing
    return -(a + b + c + hparam_prior)


res = optimize.minimize(
    map_estimate,
    x0=[2, 12, 1, 1,1 , 0.1],
    # method="L-BFGS-B",
    tol = 1e-10
)

mean, cov = get_mean_and_cov(res.x)
res.x, np.mean((data["True tide height (m)"] - mean) ** 2)

/Users/theo/Library/Caches/pypoetry/virtualenvs/aims-vYD_rrQz-py3.9/lib/python3.9/site-packages/numpy/linalg/_linalg.py:2371: RuntimeWarning:

overflow encountered in det

/Users/theo/Library/Caches/pypoetry/virtualenvs/aims-vYD_rrQz-py3.9/lib/python3.9/site-packages/scipy/optimize/_numdiff.py:590: RuntimeWarning:

invalid value encountered in subtract



(array([ 2.35519149, 12.36125313,  1.45645717,  0.25632499,  0.96446688,
         0.17498636]),
 np.float64(0.004316127518677951))

In [18]:
fig = plot_gp(x, mean, np.diag(cov)**0.5)
fig.add_trace(
    go.Scatter(x=x, y=data["True tide height (m)"], name="True tide height (m)")
)
fig.update_layout(xaxis_title='Hour', yaxis_title='Tide height (m)', title= 'Predictions with optimised hyperparameters with MAP')

# Question 6-8

In [19]:
def get_sequential_pred(obs_idx):
    t_s = t[:obs_idx]
    y_s = y[:obs_idx]
    K = kernel_function(x, x)
    Kxx = kernel_function(t_s, t_s)
    Kx = kernel_function(x, t_s)
    Kxx_inv = np.linalg.pinv(Kxx)

    # mean and cov of conditioned GP
    mean = Kx @ Kxx_inv @ y_s
    cov = K - Kx @ Kxx_inv @ Kx.T

    return mean, cov
len(x)

1258

In [20]:
idx = 800
mean, cov = get_sequential_pred(idx)
fig = plot_gp(x, mean, abs(np.diag(cov))**0.5)
fig.add_trace(
    go.Scatter(x=x, y=data["True tide height (m)"], name="True tide height (m)")
)
fig.update_layout(xaxis_title='Hour', yaxis_title='Tide height (m)', title=f'Predictions with data up to index {idx}')